In [1]:
library(qs)
library(Seurat)
library(CellAnnotatoR)
library(scCustomize)
library(conos)
library(tidyverse)
RhpcBLASctl::blas_set_num_threads(1)

qs v0.25.4.

Loading required package: SeuratObject

Loading required package: sp


Attaching package: ‘SeuratObject’


The following object is masked from ‘package:base’:

    intersect


scCustomize v2.0.1.9013
If you find the scCustomize useful please cite.
See 'samuel-marsh.github.io/scCustomize/articles/FAQ.html' for citation info.

Loading required package: Matrix

Loading required package: igraph


Attaching package: ‘igraph’


The following object is masked from ‘package:Seurat’:

    components


The following objects are masked from ‘package:stats’:

    decompose, spectrum


The following object is masked from ‘package:base’:

    union


── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.2     ✔ purrr   0.3.4
✔ tibble  3.2.1     ✔ dplyr   1.1.2
✔ tidyr   1.2.1     ✔ stringr 1.4.1
✔ readr   2.1.2     ✔ forcats 0.5.2
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::as_data_frame() masks tibbl

In [2]:
mat = Read_CellBender_h5_Multi_File(data_dir = '/data/petar/fgf1/cellbender/raw', custom_name = '.h5', merge = TRUE, parallel = TRUE, num_cores = 20)
mat %>% dim

Reading Cell Bender H5 files from directory
NOTE: Progress bars not currently supported for parallel processing.
NOTE: Parallel processing will not report informative error messages.
If function fails set `parallel = FALSE` and re-run for informative error
reporting.
Preparing & merging matrices.
Creating final sparse matrix.


[1]    32181 47030724

In [3]:
mat %>% colnames %>% head

[1] "SI-TT-A11_out_AAACCCAAGAAACACT-1" "SI-TT-A11_out_AAACCCAAGAAACCAT-1"
[3] "SI-TT-A11_out_AAACCCAAGAAACCCG-1" "SI-TT-A11_out_AAACCCAAGAAACCTG-1"
[5] "SI-TT-A11_out_AAACCCAAGAAACGAA-1" "SI-TT-A11_out_AAACCCAAGAAACTCA-1"

In [4]:
new_colnames = mat %>%
colnames %>%
str_replace('_out_', '_') %>% 
str_replace('-1', '')

In [5]:
colnames(mat) = new_colnames

In [6]:
obj = CreateSeuratObject(counts = mat, names.field = 1, names.delim = "_",
                         min.features = 1000, min.cells = 10)


In [7]:
obj

An object of class Seurat 
27757 features across 156884 samples within 1 assay 
Active assay: RNA (27757 features, 0 variable features)
 1 layer present: counts

In [8]:
obj_metadata = obj %>% 
`[[` %>% 
mutate(rowname = rownames(.)) %>%
separate(rowname, into = c("Index.10x", "barcode"), sep = "_", remove=FALSE) %>%
select(-orig.ident)

obj_metadata %>% head

,nCount_RNA,nFeature_RNA,rowname,Index.10x,barcode
,<dbl>,<int>,<chr>,<chr>,<chr>
SI-TT-A11_AAACCCAAGAATTGCA,7964,3821,SI-TT-A11_AAACCCAAGAATTGCA,SI-TT-A11,AAACCCAAGAATTGCA
SI-TT-A11_AAACCCAAGGTGGTTG,5427,2686,SI-TT-A11_AAACCCAAGGTGGTTG,SI-TT-A11,AAACCCAAGGTGGTTG
SI-TT-A11_AAACCCACAAAGGGTC,4655,2843,SI-TT-A11_AAACCCACAAAGGGTC,SI-TT-A11,AAACCCACAAAGGGTC
SI-TT-A11_AAACCCACATACAGCT,6794,3191,SI-TT-A11_AAACCCACATACAGCT,SI-TT-A11,AAACCCACATACAGCT
SI-TT-A11_AAACCCACATGAGGGT,13576,5516,SI-TT-A11_AAACCCACATGAGGGT,SI-TT-A11,AAACCCACATGAGGGT
SI-TT-A11_AAACCCAGTACGTTCA,3849,2247,SI-TT-A11_AAACCCAGTACGTTCA,SI-TT-A11,AAACCCAGTACGTTCA


In [9]:
# Set the path of the folder containing the RDS files
folder_path <- "/data/petar/fgf1/cellbender/raw"

# List all .rds files in the folder
file_names <- list.files(path = folder_path, pattern = "\\.rds$", full.names = TRUE)

# Function to extract the name from the file name
extract_name <- function(file_name) {
  # Extract the part of the file name right before "_seurat"
  sub(".*_([[:alnum:]]+)_seurat\\.rds", "\\1", basename(file_name))
}

read_meta = function(file_name){
    seurat_obj = readRDS(file_name)
    meta = seurat_obj@meta.data
    meta
}

# Read all RDS files and store them in a list with extracted names
rds_list <- setNames(lapply(file_names, read_meta), sapply(file_names, extract_name))


In [15]:
test

In [22]:
combined_metadata <- rds_list %>%
  bind_rows %>%
# filter(HTO_mcl_classification.global != 'Doublet') %>%
# filter(HTO_mcl_margin > 0.5) %>%
mutate(barcode = rownames(.)) %>%
select(-orig.ident) %>%
mutate(rowname = paste0(Index.10x, '_', barcode)) %>%
tibble %>% data.frame %>%
column_to_rownames %>%
  rename_with(~ paste0(., "_SCOP"))



combined_metadata %>% dim
combined_metadata %>% head

[1] 178706     20

,nCount_RNA_SCOP,nFeature_RNA_SCOP,Index.10x_SCOP,nCount_HTO_SCOP,nFeature_HTO_SCOP,Index.HTO_SCOP,nCount_spliced_SCOP,nFeature_spliced_SCOP,nCount_unspliced_SCOP,nFeature_unspliced_SCOP,HTO_mcl_maxID_SCOP,HTO_mcl_secondID_SCOP,HTO_mcl_margin_SCOP,HTO_mcl_classification_SCOP,HTO_mcl_classification.global_SCOP,hash.mcl.ID_SCOP,doublet_SCOP,predicted_dub_std_SCOP,predicted_dub_cut_SCOP,barcode_SCOP
,<dbl>,<int>,<chr>,<dbl>,<int>,<chr>,<dbl>,<int>,<dbl>,<int>,<chr>,<chr>,<dbl>,<chr>,<chr>,<fct>,<lgl>,<lgl>,<lgl>,<chr>
SI-TT-A11_CGCATGGAGTCGCCAC,71764,7931,SI-TT-A11,2372,12,D706,10520,3874,61244,6802,N7,M8,2.278358e-06,M8_N7,Doublet,Doublet,TRUE,FALSE,TRUE,CGCATGGAGTCGCCAC
SI-TT-A11_CCAAGCGCATCGATCA,70598,7880,SI-TT-A11,1675,12,D706,9825,3866,60773,6762,M18,M4,8.744916e-11,M18_M4,Doublet,Doublet,TRUE,FALSE,TRUE,CCAAGCGCATCGATCA
SI-TT-A11_CACATGACAGCCCACA,71679,7966,SI-TT-A11,1658,12,D706,10558,3976,61121,6808,M4,M18,3.070838e-07,M18_M4,Doublet,Doublet,TRUE,FALSE,TRUE,CACATGACAGCCCACA
SI-TT-A11_CTGCAGGAGATTCGAA,73832,8011,SI-TT-A11,2233,12,D706,9664,3883,64168,6899,J11,M8,1.780507e-01,J11_M8,Doublet,Doublet,TRUE,FALSE,TRUE,CTGCAGGAGATTCGAA
SI-TT-A11_GTTCCGTGTGAGAACC,73320,7446,SI-TT-A11,1876,12,D706,8124,3357,65196,6472,J2,M13,1.267837e-04,J2_M13,Doublet,Doublet,TRUE,FALSE,TRUE,GTTCCGTGTGAGAACC
SI-TT-A11_CGTAGTAGTTTCAGAC,74781,8014,SI-TT-A11,2023,12,D706,9984,3865,64797,6952,M2,M18,0.000000e+00,M18_M2,Doublet,Doublet,TRUE,FALSE,TRUE,CGTAGTAGTTTCAGAC


In [23]:
combined_metadata %>% colnames

[1] "nCount_RNA_SCOP"                    "nFeature_RNA_SCOP"                 
 [3] "Index.10x_SCOP"                     "nCount_HTO_SCOP"                   
 [5] "nFeature_HTO_SCOP"                  "Index.HTO_SCOP"                    
 [7] "nCount_spliced_SCOP"                "nFeature_spliced_SCOP"             
 [9] "nCount_unspliced_SCOP"              "nFeature_unspliced_SCOP"           
[11] "HTO_mcl_maxID_SCOP"                 "HTO_mcl_secondID_SCOP"             
[13] "HTO_mcl_margin_SCOP"                "HTO_mcl_classification_SCOP"       
[15] "HTO_mcl_classification.global_SCOP" "hash.mcl.ID_SCOP"                  
[17] "doublet_SCOP"                       "predicted_dub_std_SCOP"            
[19] "predicted_dub_cut_SCOP"             "barcode_SCOP"

In [25]:
obj = obj %>% AddMetaData(combined_metadata)

In [26]:
obj %>% `[[` %>% head

,orig.ident,nCount_RNA,nFeature_RNA,nCount_RNA_SCOP,nFeature_RNA_SCOP,Index.10x_SCOP,nCount_HTO_SCOP,nFeature_HTO_SCOP,Index.HTO_SCOP,nCount_spliced_SCOP,⋯,HTO_mcl_maxID_SCOP,HTO_mcl_secondID_SCOP,HTO_mcl_margin_SCOP,HTO_mcl_classification_SCOP,HTO_mcl_classification.global_SCOP,hash.mcl.ID_SCOP,doublet_SCOP,predicted_dub_std_SCOP,predicted_dub_cut_SCOP,barcode_SCOP
,<fct>,<dbl>,<int>,<dbl>,<int>,<chr>,<dbl>,<int>,<chr>,<dbl>,⋯,<chr>,<chr>,<dbl>,<chr>,<chr>,<fct>,<lgl>,<lgl>,<lgl>,<chr>
SI-TT-A11_AAACCCAAGAATTGCA,SI-TT-A11,7964,3821,23852,5466,SI-TT-A11,1929,12,D706,4342,⋯,N7,M8,0.06946145,M8_N7,Doublet,Doublet,TRUE,FALSE,TRUE,AAACCCAAGAATTGCA
SI-TT-A11_AAACCCAAGGTGGTTG,SI-TT-A11,5427,2686,18589,4492,SI-TT-A11,1233,12,D706,2936,⋯,J2,M18,0.98234971,J2,Singlet,J2,FALSE,FALSE,FALSE,AAACCCAAGGTGGTTG
SI-TT-A11_AAACCCACAAAGGGTC,SI-TT-A11,4655,2843,12818,4508,SI-TT-A11,3705,12,D706,2767,⋯,M24,M2,0.02885499,Negative,Negative,Negative,FALSE,FALSE,FALSE,AAACCCACAAAGGGTC
SI-TT-A11_AAACCCACATACAGCT,SI-TT-A11,6794,3191,22352,4816,SI-TT-A11,1000,12,D706,2976,⋯,M8,N7,0.87298667,M8,Singlet,M8,FALSE,FALSE,FALSE,AAACCCACATACAGCT
SI-TT-A11_AAACCCACATGAGGGT,SI-TT-A11,13576,5516,44099,7365,SI-TT-A11,12215,12,D706,7888,⋯,M24,M2,0.71565953,M24,Singlet,M24,FALSE,FALSE,TRUE,AAACCCACATGAGGGT
SI-TT-A11_AAACCCAGTACGTTCA,SI-TT-A11,3849,2247,17749,4347,SI-TT-A11,1699,12,D706,2474,⋯,M17,M4,0.96456947,M17,Singlet,M17,FALSE,FALSE,FALSE,AAACCCAGTACGTTCA


In [42]:
selected_cells = obj %>%
`[[` %>%
filter(HTO_mcl_classification.global_SCOP == 'Singlet') %>%
filter(HTO_mcl_margin_SCOP > 0.1) %>%
rownames

In [44]:
obj = obj %>% subset(cells = selected_cells)
obj

An object of class Seurat 
27757 features across 126957 samples within 1 assay 
Active assay: RNA (27757 features, 0 variable features)
 1 layer present: counts

In [49]:
more_meta = read_csv('../data/meta/metadata.csv') %>%
rename(hash.mcl.ID_SCOP = sample_name)
more_meta %>% head

Rows: 36 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (6): sample_name, strain, treatment, time, isolation_date, batch
dbl (1): collection_date

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


hash.mcl.ID_SCOP,strain,treatment,time,collection_date,isolation_date,batch
<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>
M9,obob,FGF1,Day5,220329,22/6/2022,Batch 1
M10,obob,FGF1,Day5,220328,23/6/2022,Batch 2
M15,obob,FGF1,Day5,220328,24/6/2022,Batch 3
N7,obob,FGF1,Day5,211213,22/6/2022,Batch 1
N17,obob,FGF1,Day5,211213,23/6/2022,Batch 2
N21,obob,FGF1,Day5,211213,24/6/2022,Batch 3


In [53]:
meta = obj %>% `[[` %>% rownames_to_column %>% left_join(more_meta, by = 'hash.mcl.ID_SCOP') %>% column_to_rownames

meta %>% head

,orig.ident,nCount_RNA,nFeature_RNA,nCount_RNA_SCOP,nFeature_RNA_SCOP,Index.10x_SCOP,nCount_HTO_SCOP,nFeature_HTO_SCOP,Index.HTO_SCOP,nCount_spliced_SCOP,⋯,doublet_SCOP,predicted_dub_std_SCOP,predicted_dub_cut_SCOP,barcode_SCOP,strain,treatment,time,collection_date,isolation_date,batch
,<fct>,<dbl>,<int>,<dbl>,<int>,<chr>,<dbl>,<int>,<chr>,<dbl>,⋯,<lgl>,<lgl>,<lgl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>
SI-TT-A11_AAACCCAAGGTGGTTG,SI-TT-A11,5427,2686,18589,4492,SI-TT-A11,1233,12,D706,2936,⋯,FALSE,FALSE,FALSE,AAACCCAAGGTGGTTG,obob,Veh_PF,Day14,220201,22/6/2022,Batch 1
SI-TT-A11_AAACCCACATACAGCT,SI-TT-A11,6794,3191,22352,4816,SI-TT-A11,1000,12,D706,2976,⋯,FALSE,FALSE,FALSE,AAACCCACATACAGCT,obob,Veh_PF,Day5,220329,22/6/2022,Batch 1
SI-TT-A11_AAACCCACATGAGGGT,SI-TT-A11,13576,5516,44099,7365,SI-TT-A11,12215,12,D706,7888,⋯,FALSE,FALSE,TRUE,AAACCCACATGAGGGT,obob,Veh_PF,Day14,220407,22/6/2022,Batch 1
SI-TT-A11_AAACCCAGTACGTTCA,SI-TT-A11,3849,2247,17749,4347,SI-TT-A11,1699,12,D706,2474,⋯,FALSE,FALSE,FALSE,AAACCCAGTACGTTCA,BL6,Veh_PF,Day14,220407,22/6/2022,Batch 1
SI-TT-A11_AAACCCAGTATCGGTT,SI-TT-A11,1530,1173,4761,2296,SI-TT-A11,1150,12,D706,944,⋯,FALSE,FALSE,FALSE,AAACCCAGTATCGGTT,BL6,Veh_PF,Day14,220407,22/6/2022,Batch 1
SI-TT-A11_AAACCCATCCAAGCAT,SI-TT-A11,6044,3023,21805,4803,SI-TT-A11,2167,12,D706,3573,⋯,FALSE,FALSE,FALSE,AAACCCATCCAAGCAT,obob,FGF1,Day5,211213,22/6/2022,Batch 1


In [54]:
obj = obj %>% AddMetaData(meta)
obj

An object of class Seurat 
27757 features across 126957 samples within 1 assay 
Active assay: RNA (27757 features, 0 variable features)
 1 layer present: counts

In [56]:
obj %>% `[[` %>% head

,orig.ident,nCount_RNA,nFeature_RNA,nCount_RNA_SCOP,nFeature_RNA_SCOP,Index.10x_SCOP,nCount_HTO_SCOP,nFeature_HTO_SCOP,Index.HTO_SCOP,nCount_spliced_SCOP,⋯,doublet_SCOP,predicted_dub_std_SCOP,predicted_dub_cut_SCOP,barcode_SCOP,strain,treatment,time,collection_date,isolation_date,batch
,<fct>,<dbl>,<int>,<dbl>,<int>,<chr>,<dbl>,<int>,<chr>,<dbl>,⋯,<lgl>,<lgl>,<lgl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>
SI-TT-A11_AAACCCAAGGTGGTTG,SI-TT-A11,5427,2686,18589,4492,SI-TT-A11,1233,12,D706,2936,⋯,FALSE,FALSE,FALSE,AAACCCAAGGTGGTTG,obob,Veh_PF,Day14,220201,22/6/2022,Batch 1
SI-TT-A11_AAACCCACATACAGCT,SI-TT-A11,6794,3191,22352,4816,SI-TT-A11,1000,12,D706,2976,⋯,FALSE,FALSE,FALSE,AAACCCACATACAGCT,obob,Veh_PF,Day5,220329,22/6/2022,Batch 1
SI-TT-A11_AAACCCACATGAGGGT,SI-TT-A11,13576,5516,44099,7365,SI-TT-A11,12215,12,D706,7888,⋯,FALSE,FALSE,TRUE,AAACCCACATGAGGGT,obob,Veh_PF,Day14,220407,22/6/2022,Batch 1
SI-TT-A11_AAACCCAGTACGTTCA,SI-TT-A11,3849,2247,17749,4347,SI-TT-A11,1699,12,D706,2474,⋯,FALSE,FALSE,FALSE,AAACCCAGTACGTTCA,BL6,Veh_PF,Day14,220407,22/6/2022,Batch 1
SI-TT-A11_AAACCCAGTATCGGTT,SI-TT-A11,1530,1173,4761,2296,SI-TT-A11,1150,12,D706,944,⋯,FALSE,FALSE,FALSE,AAACCCAGTATCGGTT,BL6,Veh_PF,Day14,220407,22/6/2022,Batch 1
SI-TT-A11_AAACCCATCCAAGCAT,SI-TT-A11,6044,3023,21805,4803,SI-TT-A11,2167,12,D706,3573,⋯,FALSE,FALSE,FALSE,AAACCCATCCAAGCAT,obob,FGF1,Day5,211213,22/6/2022,Batch 1


In [57]:
obj %>% qs::qsave('obj_cellbender.qs')

In [ ]:
rown